In [ ]:
import pandas as pd
import csv
import copy
from datetime import datetime, timedelta
from zipfile import ZipFile
from typing import Tuple, Union
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd())) # hack to get relative imports beyond the current package

In [ ]:
from ekichabi.services.android.utils import standard_format
# from ekichabi.whitelist.GetWhiteList import populate_whitelist_for_admins
def populate_whitelist_for_admins(set):
    # demo number for internal testing
    set.add(standard_format('255000000000'))

In [ ]:
# load whitelist
whitelist = {}
with open('../data/whitelisting_info.csv', 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # ignore first row - which contains field names
    next(csvreader)

    # extracting phone number(s) out of each row
    for row in csvreader:
        date = datetime.strptime(row[-1], '%Y-%d-%b')
        if date not in whitelist:
            whitelist[date] = set()
        if str(row[2]):
            whitelist[date].add(standard_format(str(row[2])))  # number1
        if str(row[3]):
            whitelist[date].add(standard_format(str(row[3])))  # number2
print(whitelist.keys())
totalwhitelist = set.union(*whitelist.values())
print(len(totalwhitelist))

In [ ]:
missingwhitelist = set()
with open('../data/whitelist12-12.csv', 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # extracting phone number(s) out of each row
    for row in csvreader:
        missingwhitelist.add(standard_format(str(row[0])))
len(missingwhitelist)

In [ ]:
# load Census Survey information
survey_df = pd.read_csv('../data/census_survey.csv')
survey_df['phone_number'] = survey_df['phone_number'].apply(lambda x: standard_format(str(x)))
survey_df.head()

In [ ]:
survey_df['phone_number'].nunique()

In [ ]:
survey_df.info()

In [ ]:
# load enumerators
enumerators = set()
with open('../data/Enumerator.csv', 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # ignore header
    next(csvreader)

    # extracting phone number out of each row
    for row in csvreader:
        enumerators.add(standard_format(str(row[2])))  # number1
print(enumerators)

In [ ]:
# load testers and admins
testers = set()
populate_whitelist_for_admins(testers)
print(testers)

In [ ]:
# load wakala intervention group
wakalagroup = set()
with open('../data/Wakala.csv', 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # extracting phone number out of each row
    for row in csvreader:
        wakalagroup.add(standard_format(str(row[1])))  # number1
print(wakalagroup)

In [ ]:
# load business group
cutoffdate = datetime.strptime('2022-7-Dec', '%Y-%d-%b')
businessgroup = set()
with open('../data/census_data_trimmed.csv', 'r') as csvfile:
    # creating a csv reader object
    csvreader = csv.reader(csvfile)

    # ignore first row - which contains field names
    next(csvreader)

    # extracting phone number out of each row
    for row in csvreader:
        businessgroup.add(standard_format(str(row[17])))  # number1
        businessgroup.add(standard_format(str(row[18])))  # number2
businessgroup.remove("")
len(businessgroup)

In [ ]:
df28 = pd.read_csv("./whitelisted_hh_phone_nums_28Nov2022 (2).csv")
df28["date"] = "2022-28-Nov"
df12 = pd.read_csv("./whitelisted_hh_phone_nums_asof12.12 (1).csv")
df12["date"] = "2022-12-Dec"
df23 = pd.read_csv("./whitelisted_hh_phone_nums_23Nov2022 (2).csv")
df23["date"] = "2022-23-Nov"
df06 = pd.read_csv("./whitelisted_hh_phone_nums_06Dec2022 (2).csv")
df06["date"] = "2022-6-Dec"
df = pd.concat([df23, df28, df06, df12], ignore_index=True)
df_drop = df.drop_duplicates(subset=["dir_phn1"], keep='first').drop(['digital','hhID'], axis=1)

In [ ]:
df_drop['source'] = 'WHITELIST_CSV'

In [ ]:
df23

In [ ]:
missing = # SECRET

In [ ]:
missing['source'] = "EXTRA_FROM_BASELINE"
missing['date'] = "2023-10-Mar"

In [ ]:
missing

In [ ]:
admin = pd.DataFrame(set.union(enumerators, testers), columns=['dir_phn1'])
admin['source'] = "ENUMERATORS_AND_TESTERS"
admin['date'] = "2022-Nov-10"
admin

In [ ]:
waka = pd.DataFrame(wakalagroup, columns=['dir_phn1'])
waka['source'] = "WAKALAS"
waka['date'] = "2022-16-Nov"
waka

In [ ]:
pd.concat([admin, df_drop, missing, waka], ignore_index=True).drop_duplicates(subset=["dir_phn1"], keep='first').to_csv('whitelist_pretty')

In [ ]:
df_drop['date'].unique()

In [ ]:
combined_whitelist = set.union(*[key for key in whitelist.values()][1:-2], missingwhitelist)
len(combined_whitelist)

In [ ]:
len(testers)

In [ ]:
len(set.difference(combined_whitelist, survey_df['phone_number']))

In [ ]:
len(set.union(combined_whitelist, survey_df['phone_number']))

In [ ]:
len(combined_whitelist & set(survey_df['phone_number']))

In [ ]:
l = [number for number in set(survey_df['phone_number']) if number not in set.union(totalwhitelist, combined_whitelist, missingwhitelist) if number not in enumerators and number not in testers]
l

In [ ]:
survey_df[survey_df['phone_number'].isin(l)].nunique()

In [ ]:
len(set([number for number in combined_whitelist if number not in set(survey_df['phone_number']) if number not in enumerators and number not in testers]))

In [ ]:
len(set(survey_df['phone_number']).difference(combined_whitelist))

In [ ]:
def wasWhitelisted(number: str, date: datetime) -> bool:
    number = standard_format(number)
    if number in testers or number in enumerators:
        return False
    if date <= cutoffdate and number in businessgroup:
        return True
    for d, l in whitelist.items():
        if d > date:
            return False
        if number in l:
            return True
    return False

In [ ]:
# load log data
def parseLine(line: bytes) -> Tuple[datetime, str, Union[str, bool]]:
    line = line.decode().strip()
    columns = line.split('\t')
    t = datetime.strptime(columns[0], '[%H:%M:%S.%f]')
    time = startTime + timedelta(hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=t.microsecond)
    action = columns[1]
    details = columns[3] if len(columns) > 3 else False
    return time, action, details

def initRow(number, startTime):
    return {'phone_number': number, 'session_start': startTime, 'session_end': pd.NaT, 'duration': pd.NaT,'inputs': "", 
        'group': 'non-whitelisted' if (not wasWhitelisted(number, startTime)) else 'wakalagroup' if number in wakalagroup else 'business' if number in businessgroup else 'household', 
        'businesses_visited': 0, 'businesses': [], 'searches': 0, 'filter_location': 0, 'helper_instruction': 0, 'category_browse': 0, 'location_browse': 0, 'exit': 0, 'home': 0, 'back': 0}

df = pd.DataFrame(columns=initRow('000000000', 0).keys())
with ZipFile('../logsArchive/all_ussd.zip', 'r') as archive:
    for name in archive.namelist(): # name = [phone_number]_[Year]-[month]-[day]-[Hour].[Minute].[Second]
        parts = name.split('_')
        number = standard_format(parts[0])
        print(name)
        print(parts[1].replace('.log', ''))
        startTime = datetime.strptime(parts[1].replace('.log', ''), '%Y-%m-%d-%H.%M.%S')
        # if not wasWhitelisted(number, startTime):
        #     continue
        if number in testers or number in enumerators:
            continue
        row = initRow(number, startTime)
        isHome = True
        path = []
        with archive.open(name) as file:
            file.readline() # ignore session info
            line = file.readline() # ignore column headers
            # line = file.readline() # ignore first home screen

            def handleReadLine():
                global line
                global row
                global isHome
                global path
                line = file.readline()
                if not line:
                    return "", "", ""
                time, action, details = parseLine(line)
                row['session_end'] = time
                row['duration'] = time - startTime
                if action == 'INPUT RECEIVED':
                    row['inputs'] += ("," if len(row['inputs']) != 0 else "") + details.strip("'")
                elif action == 'RENDERED SCREEN' and details == 'SearchSelectorScreen':
                    row['searches'] += 1
                elif action == 'RENDERED SCREEN' and details == 'IfFilterBYLocationScreen':
                    row['filter_location'] += 1
                elif action == 'RENDERED SCREEN' and details == 'HelpScreen':
                    row['helper_instruction'] += 1
                elif isHome and action == 'RENDERED SCREEN' and details == 'MenuHierarchyScreen - selecting a category':
                    row['category_browse'] += 1
                elif isHome and action == 'RENDERED SCREEN' and details == 'MenuHierarchyScreen - selecting a district':
                    row['category_browse'] += 1
                elif action == 'RENDERED SCREEN' and details and details.startswith('BusinessDetailsScreen'):
                    row['businesses_visited'] += 1
                    duration = pd.NaT
                    next_line = file.peek()
                    if next_line:
                        next_time, _, _ = parseLine(next_line)
                        duration = next_time - time
                    business = {'id': details.split(' - for ')[1], 'duration': duration,'path': copy.copy(path)}
                    row['businesses'].append(business)
                if action == 'RENDERED SCREEN':
                    path.append(details)
                    isHome = False
                return time, action, details

            while line:
                time, action, details = handleReadLine()
                if not line: break
                if action == 'BACK PRESSED': # BACK PRESSED + RENDERED SCREEN (back button -- 99)
                    row['back'] += 1
                    _, _, details = handleReadLine()
                    isHome = details == 'HomeScreen'
                    if isHome:
                        path = []
                elif action == 'HOME PRESSED': # HOME PRESSED + RENDERED SCREEN (home button -- 100)
                    row['home'] += 1
                    handleReadLine()
                    isHome = True
                    path = []
                elif action == 'EXIT PRESSED': # EXIT PRESSED + RENDERED SCREEN (session end button -- 109)
                    row['exit'] += 1
                    handleReadLine()
        df = pd.concat([df, pd.DataFrame(row.values(), index=row.keys()).T], ignore_index=True)
        # break

    print(len(archive.namelist()))
pd.set_option('display.max_colwidth', None)
df.head()

In [ ]:
df.info()

In [ ]:
def categorizeGroup(number):
    return 'wakalagroup' if number in wakalagroup else 'business' if number in businessgroup else 'household' 

In [ ]:
df.loc[df['group'] == 'non-whitelisted']['phone_number'].nunique()

In [ ]:
len(set(df.loc[df['group'] == 'non-whitelisted']['phone_number']) & set.union(*whitelist.values()))

In [ ]:
df.loc[~pd.isnull(df['session_end']) & (df['inputs'] != "") & (df['group'] == "non-whitelisted")].sort_values(by='session_end')

In [ ]:
with ZipFile('../logsArchive/all_ussd.zip', 'r') as archive:
    print(archive.read('SECRET_2023-01-24-05.09.05.log'))

In [ ]:
def label_function(val):
    return f'{val:.0f}\n{val / len(df) * 100:.0f}%'

groups = df.loc[~(df['group'] == "non-whitelisted")].groupby('group').size()
patches, texts = plt.pie(groups, textprops={'fontsize': 20}, colors=['tomato', 'gold', 'skyblue'])
labels = [group + "-" + label_function(val) for val, group in zip(groups, ['business', 'household', 'wakalagroup'])]
plt.legend(patches, labels, loc='center left', bbox_to_anchor=(-0.1, 1.), fontsize=8)
plt.title("Distribution of sessions")

In [ ]:
groups = df.loc[~(df['group'] == "non-whitelisted")].groupby('group')['phone_number'].nunique()
patches, texts = plt.pie(groups, textprops={'fontsize': 20}, colors=['tomato', 'gold', 'skyblue'])
labels = [group + "-" + label_function(val) for val, group in zip(groups, ['business', 'household', 'wakalagroup'])]
plt.legend(patches, labels, loc='center left', bbox_to_anchor=(-0.1, 1.), fontsize=8)
plt.title("Distribution of numbers")

In [ ]:
groups = df.loc[~(df['group'] == "non-whitelisted")].groupby('group').size() / df.loc[~(df['group'] == "non-whitelisted")].groupby('group')['phone_number'].nunique()
patches, texts = plt.pie(groups, textprops={'fontsize': 20}, colors=['tomato', 'gold', 'skyblue'])
labels = [group + "-" + label_function(val) for val, group in zip(groups, ['business', 'household', 'wakalagroup'])]
plt.legend(patches, labels, loc='center left', bbox_to_anchor=(-0.1, 1.), fontsize=8)
plt.title("Distribution of Sessions/Number")

In [ ]:
(df.groupby(['group', 'searches']).size() / df.groupby('group').size()).unstack().plot(kind='bar', stacked=True)

In [ ]:
(df.groupby(['group', 'businesses_visited']).size() / df.groupby('group').size()).unstack().plot(kind='bar', stacked=True)

In [ ]:
(df.groupby(['group', 'category_browse']).size() / df.groupby('group').size()).unstack().plot(kind='bar', stacked=True)

In [ ]:
(df.groupby(['group', 'location_browse']).size() / df.groupby('group').size()).unstack().plot(kind='bar', stacked=True)

In [ ]:
(df.groupby(['group', 'back']).size() / df.groupby('group').size()).unstack().plot(kind='bar', stacked=True)

In [ ]:
(df.groupby(['group', 'exit']).size() / df.groupby('group').size()).unstack().plot(kind='bar', stacked=True)

In [ ]:
(df.groupby(['group', 'home']).size() / df.groupby('group').size()).unstack().plot(kind='bar', stacked=True)

In [ ]:
(df.groupby(['group', 'filter_location']).size() / df.groupby('group').size()).unstack().plot(kind='bar', stacked=True)

In [ ]:
df['duration'] = df['session_end'] - df['session_start']
df['duration'] = df['duration'].astype('timedelta64[s]')
fig, ax = plt.subplots(figsize=(8,5))
for label, df1 in df.groupby('group'):
     df1.duration.plot(kind="kde", ax=ax, label=label)
ax.set_xlabel("Session Duration (seconds)")
ax.set_xlim(0, 800)
ax.legend(loc="upper left")

In [ ]:
# sessions per phone number
df.groupby('group').size() / df.groupby('group')['phone_number'].nunique()

In [ ]:
df['time_after'] = (df['session_start'] - datetime(2022,11,1,0,0)) / np.timedelta64(1,'D')
fig, ax = plt.subplots(figsize=(8,5))
for label, df1 in df.groupby('group'):
     df1.time_after.plot(kind="kde", ax=ax, label=label)
ax.set_xlabel("Session start date after November 1st")
ax.set_xlim(0, 290)
ax.legend(loc="upper left")